In [1]:
import pandas as pd
import numpy as np
import os
import importlib
import random
import pickle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, average_precision_score
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForMaskedLM, BertConfig, BertModel, InputExample
from run_classifier_dataset_utils import convert_examples_to_features, parallel_convert_examples_to_features
from tqdm.auto import tqdm
from torch.utils import data
from scipy import stats
import matplotlib.pyplot as plt
import math

pd.set_option('display.max_columns', 50)
pd.options.mode.chained_assignment = None

/data/anaconda/envs/ml4h/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/anaconda/envs/ml4h/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/anaconda/envs/ml4h/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/anaconda/envs/ml4h/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

## Preliminaries

Run the `create_data.py` script to generate the following files:

- `cohort.csv`:
  - Contains one record for each adult patient’s first ICU stay over 48 hours in lengthwithin their first hospital admission.
  - The `mort_icu` column represents whether the patient died during their ICU stay.
  - The columns from `Acute Renal` to `Shock` correspond to each of the 25 CCS code  groups, which are derived from  ICD-9 codes assigned at the end of a patient’s hospital stay.
  - The `Any Acute` and `Any Chronic` columns are derived from whether the patient has any acute and chronic phenotypes respectively.
  
  
- `notes.csv`
  - Contains, for each of the patients in `cohort.csv`, all of the notes written during their hospital stay (along with the timestamp) for the following note types:
      - Discharge Summary
      - Nursing
      - Nursing/other
  - The notes have been lightly preprocessed (ex: removing PHI identifiers, removing section numbers).

In [3]:
data_path = 'mimic_data/'
bert_path = 'section2_pre/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000/'
# bert_path = '/scratch/gobi1/haoran/shared_data/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000/'
#/data/home/linda/ml4h_workspace/Problem_Set_2/section2_pre/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
model = BertForMaskedLM.from_pretrained(bert_path).to(device)
tokenizer = BertTokenizer.from_pretrained(bert_path)

In [47]:
cohort = pd.read_hdf(os.path.join(data_path, 'cohort.h5'))
notes = pd.read_hdf(os.path.join(data_path, 'notes.h5'))

In [7]:
cohort.head()

,subject_id,hadm_id,icustay_id,gender,admittime,dischtime,age,ethnicity,admission_type,language,insurance,hospital_expire_flag,mort_icu,intime,Acute Renal,Cerebrovascular,Myocardial,Dysrhythmias,Chronic Kidney,COPD,Comp. Surgical,Conduction,Heart Failure,Atherosclerosis,Diabetes Comp,Diabetes No Comp,Lipid Metabolism,Hypertension,Fluid Disorder,GI Hemorrhage,Hypertension Comp,Other Liver,Lower Resp,Upper Resp,Pleurisy,Pneumonia,Resp Failure,Septicemia,Shock,Any Acute,Any Chronic,train
0,3,145834,211552,M,2101-10-20 19:08:00,2101-10-31 13:58:00,76.0,white,EMERGENCY,Missing,Medicare,0,0,2101-10-20 19:10:11,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1
1,6,107064,228232,F,2175-05-30 07:15:00,2175-06-15 16:00:00,65.0,white,ELECTIVE,English,Medicare,0,0,2175-05-30 21:30:54,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
2,9,150750,220597,M,2149-11-09 13:06:00,2149-11-14 10:15:00,41.0,other,EMERGENCY,Missing,Medicaid,1,1,2149-11-09 13:07:02,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0
3,12,112213,232669,M,2104-08-07 10:15:00,2104-08-20 02:57:00,72.0,white,ELECTIVE,Missing,Medicare,1,0,2104-08-08 02:08:17,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1
4,13,143045,263738,F,2167-01-08 18:43:00,2167-01-15 15:15:00,39.0,white,EMERGENCY,Missing,Medicaid,0,0,2167-01-08 18:44:25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [8]:
cohort.shape

(19392, 42)

In [9]:
notes.head()

,note_id,subject_id,hadm_id,chartdate,charttime,category,text
0,174,22532,167853.0,2151-08-04,NaT,Discharge summary,service: addendum: radiologic s...
1,175,13702,107527.0,2118-06-14,NaT,Discharge summary,date of birth: ...
4,178,26880,135453.0,2162-03-25,NaT,Discharge summary,date of birth: ...
6,180,20646,134727.0,2112-12-10,NaT,Discharge summary,service: medicine aller...
7,181,42130,114236.0,2150-03-01,NaT,Discharge summary,date of birth: ...


In [10]:
notes.shape

(425549, 7)

## Sentence Completion

In [11]:
def fill_blank(text: str, model: BertForMaskedLM, tokenizer: BertTokenizer) -> (str, dict):
    '''
    Given a sentence with a single blank (denoted by an underscore), queries the BERT model to 
        fill in the missing token.
        
    Inputs:
        - text: sentence containing a single underscore corresponding to the missing token
                ex: "[CLS] 40 yo asian homeless man with h/o polysubstance abuse and recently released from _  [SEP]"
        - model: pytorch ClinicalBERT model, of type BertForMaskedLM
        - tokenizer: BertTokenizer object
    
    Output:
        - tuple consisting of the following:
            - string corresponding to the sentence where the underscore is replaced with the most likely token
                ex: "[CLS] 40 yo asian homeless man with h / o polysubstance abuse and recently released from home [SEP]"
            - a dictionary str:float mapping each word in the vocabulary to its normalized probability.
                - sum of the values should be equal to 1
                - the dictionary should have 28996 elements
    '''
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    
    # very important!!!
    text = text.replace('_', "[MASK]")
    
    #https://stackoverflow.com/questions/54978443/predicting-missing-words-in-a-sentence-natural-language-processing-model
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens]).cuda()
    segments_tensors = torch.tensor([segments_ids]).cuda()

    # Load pre-trained model (weights)
    model.eval()

    masked_index = tokenized_text.index('[MASK]')
    
    # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    # Only for predict the missing word
    predicted_index = torch.argmax(predictions[0][0][masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    full_text = text.replace("[MASK]", predicted_token)

    prob_dict = {}

    # https://github.com/huggingface/transformers/issues/547
    # Calculate the probability
    top_k = 28996      # top 28996 will sum up to 1
    probs = torch.nn.functional.softmax(predictions[0][0][masked_index], dim = 0)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted = True)    #Note that here is "False" (whether or not sort by prob)

    prob_dict = {}

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        prob_dict[predicted_token] = float(token_weight)

    return full_text, prob_dict

In [12]:
def test_fill_blank():
    text = '[CLS] 40 yo asian homeless man with h/o polysubstance abuse and recently released from _ [SEP]'
    a,b = fill_blank(text, model, tokenizer)
    assert(a.split(' ')[-2] == 'home'), 'Most likely word not correct!'
    assert(math.isclose(np.sum(list(b.values())), 1.0, rel_tol = 1e-4)), 'Probabilities not normalized!'
    assert(math.isclose(b['shelter'], 0.021500807255506516, rel_tol = 1e-4)), "Probability not correct!"
    print("Test passed!")
    
test_fill_blank()

Test passed!


In [13]:
races = ['caucasian', 'asian',  'hispanic', 'black']
text = '[CLS] 40 yo [RACE] homeless man with h/o polysubstance abuse and recently released from _ [SEP]'
for r in races:
    print(fill_blank(text.replace('[RACE]', r), model, tokenizer)[0])

[CLS] 40 yo caucasian homeless man with h/o polysubstance abuse and recently released from home [SEP]
[CLS] 40 yo asian homeless man with h/o polysubstance abuse and recently released from home [SEP]
[CLS] 40 yo hispanic homeless man with h/o polysubstance abuse and recently released from home [SEP]
[CLS] 40 yo black homeless man with h/o polysubstance abuse and recently released from prison [SEP]


In [14]:
# 2b here
#this might involve for different ethnicities, a difference in prescribed treatment, prognosis, or propagation of social stereotypes.
# this is an 81-year-old female with a history of emphysema (not on home o2), who presents with three days of shortness of breath thought by her primary care doctor to be a copd flare.  
#she was started on a prednisone taper and one day prior to admission she required oxygen at home in order to maintain oxygen saturation greater than 90%.

races = ['caucasian', 'asian',  'hispanic', 'black']
text = '[CLS] this is an 81-year-old male [RACE] with a history of _ (not on home o2), who presents with three days of shortness of breath. [SEP]'
for r in races:
    print(fill_blank(text.replace('[RACE]', r), model, tokenizer)[0])

[CLS] this is an 81-year-old male caucasian with a history of diabetes (not on home o2), who presents with three days of shortness of breath. [SEP]
[CLS] this is an 81-year-old male asian with a history of falls (not on home o2), who presents with three days of shortness of breath. [SEP]
[CLS] this is an 81-year-old male hispanic with a history of falls (not on home o2), who presents with three days of shortness of breath. [SEP]
[CLS] this is an 81-year-old male black with a history of diabetes (not on home o2), who presents with three days of shortness of breath. [SEP]


## Log Probability Scores

In [15]:
gender_words = {'male': ['man', 'he', 'male', 'm'],
                'female': ['woman', 'she', 'female', 'f']}

In [16]:
def log_prob_score(text: str, target_word: str, attribute_word:str, model: BertForMaskedLM, tokenizer: BertTokenizer) -> float:
    '''
    Given a template sentence, an attribute, and a target word, computes the log probability bias score,
      using the method by Kurita et al.
    
    Inputs:
        - text: a template sentence, ex: '[CLS] [GEND] has a pmd of [ATTR] [SEP]'
        - target_word: a gender pronoun, ex: man
        - attribute_word: word from one of the clinical categories, ex: cvd
        - model: pytorch ClinicalBERT model, of type BertForMaskedLM
        - tokenizer: BertTokenizer object
    
    Output: float representing the log-probability score    
    
    For consistency, use the natural logarithm for the log.
    Note that the attribute_word could be multiple tokens. This should not affect your calculation.
    Hint: call fill_blank to reduce code duplication
    '''
    # You might want to call the fill blank function from the previous question.
    # log_prob_score('[GEND] has a pmd of [ATTR]', 'man', 'cvd', model, tokenizer)
    
    # four step to be followed:
    # step1 sentense has already given
    
    # step2 Replace [TARGET] with [MASK]
    # text: _ has a pmd of cvd
    text_step2 = text.replace('[GEND]', "_")
    text_step2 = text_step2.replace('[ATTR]', attribute_word)
    full_text, pred_dict = fill_blank(text_step2, model, tokenizer)
    p_tgt = pred_dict[target_word]
    
    # step3 Replace both [TARGET] and [ATTRIBUTE] with [MASK], then compute the probability BERT assigns to the sentence
    text_step3 = text.replace('[GEND]', "_")
    text_step3 = text_step3.replace('[ATTR]', "_")
    full_text, pred_dict = fill_blank(text_step3, model, tokenizer)

    # p_prior = pred_dict[list(pred_dict.keys())[0]]
    p_prior = pred_dict[target_word]
    
    # step4 ln(p_tgt / p_prior)
    score = np.log(p_tgt / p_prior)
    return score

In [17]:
def test_log_prob_score():
    assert( np.isclose(log_prob_score('[GEND] has a pmd of [ATTR]', 'man', 'cvd', model, tokenizer),
    -1.5143764856436155) , 'Test failed!')
    print("Test passed!")

test_log_prob_score()

<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-17-f2010c8be359>:2: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert( np.isclose(log_prob_score('[GEND] has a pmd of [ATTR]', 'man', 'cvd', model, tokenizer),


Test passed!


In [18]:
log_cats = {
    'cvd': ['cvd', 'cad', 'chf', 'coronary artery disease', 'coronary heart disease', 'heart failure', 
            'heart attacks', 'heart problems'],
    'hypertension': ['hypertension','high blood pressure','high bp', 'abnormally high bp',
                    'pulmonary artery systolic hypertension'],
    'diabetes': ['diabetes', 'diabetes mellitus'],
    'mental health': ['depression', 'mental health']
}

In [19]:
templates = {
    cat: ['[CLS] [GEND] has a pmd of [ATTR] [SEP]',
            '[CLS] [GEND] has a hx of [ATTR] [SEP]',
            '[CLS] [GEND] has a pmh of [ATTR] [SEP]',
            '[CLS] [GEND] with a discharge diagnosis of [ATTR] [SEP]',
            '[CLS] [GEND] was admitted with [ATTR] [SEP]'] for cat in log_cats}

In [20]:
# 3b: run this code and paste the table in your report
def evaluate_log_probs(gender_words, log_cats, templates):
    '''
    Horrendously slow because no batching and calling prior more times than necessary.
    '''
    sents = []
    for cat in log_cats:
        for attr in log_cats[cat]:                    
            for gender in gender_words:
                for pronoun in gender_words[gender]:
                    for te in templates[cat]:
                        sents.append((cat, gender, te, pronoun, attr))
    
    raw_results = []
    for s in tqdm(sents):
        sco = log_prob_score(*s[2:], model = model, tokenizer = tokenizer)
        raw_results.append((s[0], s[1], s[2], s[4], sco))    
    
    result_df = pd.DataFrame(raw_results, columns = ['category', 'gender', 'template', 'attr', 'score'])
    cat_res = []
    for cat in result_df['category'].unique():
        tmp = result_df[result_df.category == cat]
        m = tmp[tmp.gender == 'male'].score
        f = tmp[tmp.gender == 'female'].score
        sig = stats.wilcoxon(m, f)
        cat_res.append((cat, np.mean(m), np.mean(f), sig[1]))
    return pd.DataFrame(cat_res, columns = ['Category', 'Male log-prob', 'Female log-prob', 'p-value'])

log_res_df = evaluate_log_probs(gender_words, log_cats, templates)
display(log_res_df)

,Category,Male log-prob,Female log-prob,p-value
0,cvd,1.079810,0.962445,0.000002
1,hypertension,0.310041,0.263864,0.094717
2,diabetes,0.740889,0.658606,0.037215
3,mental health,0.101301,0.190988,0.242245


## Multi-Group Fairness Metric

In [21]:
import metrics
importlib.reload(metrics)

def multigroup_test():
    test_df1 = pd.DataFrame(data  = {
        'protected': ['A','B', 'C']*4,
        'target': [0]*5 + [1]*7,
        'pred':[0.1, 0.8, 0.8, 0.9, 0.3, 0.3, 0.99, 0.1, 0.1, 0.9, 0.5, 0.5]
    })
    ret = metrics.recall_gap_multigroup(test_df1, 'protected', 'target', 'pred', 0.4)
    assert(
        len(ret) == 3 and
        np.isclose(ret['A'], 0.6666666666666667) and
        np.isclose(ret['B'], -0.5) and
        np.isclose(ret['C'], -0.6666666666666667)    
    ) , 'Test failed!'
    print("Test passed!")
    
multigroup_test()

Test passed!


## Biases in Downstream Tasks

In [22]:
# reload model to output hidden states
config = BertConfig.from_pretrained(bert_path, output_hidden_states=True)
model = BertModel.from_pretrained(bert_path, config = config).to(device)
tokenizer = BertTokenizer.from_pretrained(bert_path)
n_gpu = torch.cuda.device_count()
if device == 'cuda' and n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [23]:
notes.category.value_counts()

Nursing/other        282704
Nursing              120615
Discharge summary     22230
Name: category, dtype: int64

In [24]:
print(notes.groupby(['subject_id', 'hadm_id']).agg({'text':'count'})['text'].value_counts().describe())
time_steps = 35 # choose to take most recent 35 notes for each patient

count     256.000000
mean       75.093750
std       286.161244
min         1.000000
25%         2.000000
50%         7.000000
75%        35.500000
max      3983.000000
Name: text, dtype: float64


In [25]:
notes.loc[notes.category == 'Discharge summary', 'charttime'] = notes[notes.category == 'Discharge summary'].iloc[0]['chartdate'] + pd.Timedelta(days = 1) -  pd.Timedelta(seconds = 1)
cohort = cohort.reset_index(drop = True)

def index_notes(x):
    # take most recent time_steps notes
    if len(x) > time_steps:
        return np.concatenate((-1*np.ones(len(x) - time_steps), np.arange(time_steps)))
    else:
        return np.arange(len(x))
    
notes['note_index'] = -1
notes['note_index'] = notes.sort_values(by = ['subject_id','charttime'], ascending = True).groupby('subject_id').transform(index_notes).astype(int)
notes = notes[notes.note_index >= 0] # drop excess notes > time_steps

mapping = dict(map(reversed, cohort['subject_id'].to_dict().items()))
notes.shape

(288436, 8)

In [26]:
def convert_input_example(subject_id, note_ind, text):
    return InputExample(guid = '%s-%s'%(subject_id, note_ind), text_a = text, text_b = None, label = 0)

examples = [convert_input_example(row['subject_id'], row['note_index'], row['text']) for idx, row in notes.iterrows()]

In [27]:
# will take ~(150/n_cpu) min
features = parallel_convert_examples_to_features(examples, 512, tokenizer, 
                                                 output_mode = 'classification', n_cpus = 10)

Using 10 CPUs


In [28]:
class MIMICDataset(data.Dataset):
    def __init__(self, features):
        self.features = features
        self.length = len(features)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        all_input_ids = torch.tensor(self.features[index].input_ids, dtype = torch.long)
        all_input_mask = torch.tensor(self.features[index].input_mask, dtype = torch.long)
        all_segment_ids = torch.tensor(self.features[index].segment_ids, dtype = torch.long)
        y = torch.tensor(self.features[index].label_id, dtype = torch.float32)
        guid = self.features[index].guid

        return all_input_ids, all_input_mask, all_segment_ids, y, guid

def extract_embeddings(v):
    return torch.cat((v[-1][:, 0, :] , v[-2][:, 0, :] , v[-3][:, 0, :] , v[-4][:, 0, :]), 1)

def get_embs(generator):
    model.eval()
    with torch.no_grad():
        for input_ids, input_mask, segment_ids, _,  guid in tqdm(generator):
            input_ids = input_ids.to(device)
            segment_ids = segment_ids.to(device)
            input_mask = input_mask.to(device)
            hidden_states = model(input_ids, token_type_ids = segment_ids, attention_mask = input_mask)[2]
            bert_out = extract_embeddings(hidden_states)
                        
            for c,i in enumerate(guid):
                sub_id, note_idx = i.split('-')
                embs = bert_out[c,:].detach().cpu()
                inputs[mapping[int(sub_id)], int(note_idx), :] = embs
    return True

emb_dim = 768*4

In [29]:
# inputs = torch.zeros((cohort.shape[0], time_steps, emb_dim)) # num_patient x time_steps x emb_size
# data_generator = data.DataLoader(MIMICDataset(features), shuffle = True,  batch_size = 64*n_gpu)

# # will take several hours
# get_embs(data_generator)

In [30]:
## might want to cache `inputs` to avoid running the above cell again
# torch.save(inputs, open('inputs.pt', 'wb'))
inputs = torch.load(open('inputs.pt', 'rb'))

In [31]:
seq_lengths = notes.sort_values(by = 'subject_id').groupby('subject_id').agg({'note_index': 'max'})['note_index'].values + 1
targets = pd.read_csv('mapping.csv')['after'].values.tolist()

In [48]:
cohort.loc[cohort['train'] == 0, 'split'] = 'test'
cohort.loc[(cohort['train'] == 1) & (np.random.rand(len(cohort)) < 0.8), 'split'] = 'train'
cohort['split'] = cohort['split'].fillna('val')

In [49]:
cohort['split'].unique()

array(['train', 'test', 'val'], dtype=object)

In [33]:
# Report the following to get the 1 point
print(inputs[35, 10, 1234])
# 5a ends here

tensor(-0.0743)


### 5 b)
To build your model, you have the following variables defined in memory:
- `targets`: list of 27 predictive targets which corresponding to column names in `cohort`
- `cohort`: dataframe containing targets. Make sure to following the train/val/test split in the `split` column
- `inputs`: num_patient x time_steps (35) x emb_size (3072 (768*4 = 3072)) tensor. Each [i, :, :] slice of the tensor corresponds to a single patient.
- `mapping`: maps the `subject_id` field to an index in the `inputs` tensor. For example, the features for the patient with subject_id=3 is at the index=0 slice of `inputs`.
- `seq_lengths`: array of size num_patient, where each element represents how many notes (up to 35) the patient had. This is the number of non-zero note embeddings that a patient has in `inputs`.

In [50]:
# 5b here
## train model: LSTM --> mort_icu

# split cohort
# y_cohort = cohort['subject_id', 'mort_icu', 'split']
# y_train_cohort = y_cohort[y_cohort['split'] == 'train']
# y_val_cohort = y_cohort[y_cohort['split'] == 'val']
# y_test_cohort = y_cohort[y_cohort['split'] == 'test']

y_cohort_df = cohort[['subject_id','Acute Renal', 'Cerebrovascular', 'Myocardial', 
                      'Dysrhythmias','Chronic Kidney', 'COPD', 'Comp. Surgical', 'Conduction',
       'Heart Failure', 'Atherosclerosis', 'Diabetes Comp', 'Diabetes No Comp',
       'Lipid Metabolism', 'Hypertension', 'Fluid Disorder', 'GI Hemorrhage',
       'Hypertension Comp', 'Other Liver', 'Lower Resp', 'Upper Resp',
       'Pleurisy', 'Pneumonia', 'Resp Failure', 'Septicemia', 'Shock',
       'Any Acute', 'Any Chronic', 'split']]

y_train_cohort_df = y_cohort_df[y_cohort_df['split'] == 'train']
y_val_cohort_df = y_cohort_df[y_cohort_df['split'] == 'val']
y_test_cohort_df = y_cohort_df[y_cohort_df['split'] == 'test']

train_idx = [mapping[item] for item in y_train_cohort_df['subject_id'].tolist()]
val_idx = [mapping[item] for item in y_val_cohort_df['subject_id'].tolist()]
test_idx = [mapping[item] for item in y_test_cohort_df['subject_id'].tolist()]

X_train_inputs_tensor = inputs[train_idx]
X_val_inputs_tensor = inputs[val_idx]
X_test_inputs_tensor = inputs[test_idx]

y_train_cohort_df = y_train_cohort_df.drop(columns = ['subject_id', 'split'])
y_val_cohort_df = y_val_cohort_df.drop(columns = ['subject_id', 'split'])
y_test_cohort_df = y_test_cohort_df.drop(columns = ['subject_id', 'split'])

In [35]:
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
# from sklearn.metrics import f1_score
from sklearn.metrics import f1_score, accuracy_score

In [52]:
lstm_model = Sequential()
lstm_model.add(LSTM(128, batch_input_shape=(None, X_train_inputs_tensor.shape[1], X_train_inputs_tensor.shape[2])))    #lstm layer
# lstm_model.add(Dropout(0.3))    #training process: 20% neurons mask to 0, but this won't happen in testing
lstm_model.add(Dense(27, activation='sigmoid'))    # dense layer(normal neural nets layer)
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# multilabel

lstm_model.fit(X_train_inputs_tensor, y_train_cohort_df,
      epochs=30, batch_size=64,
      verbose=1, shuffle=True,
      validation_data=(X_val_inputs_tensor, y_val_cohort_df))

Train on 10848 samples, validate on 2762 samples
Epoch 1/30
10848/10848 [==============================] - 21s 2ms/sample - loss: 0.4502 - acc: 0.8122 - val_loss: 0.4406 - val_acc: 0.8147
Epoch 2/30
10848/10848 [==============================] - 20s 2ms/sample - loss: 0.4364 - acc: 0.8168 - val_loss: 0.4409 - val_acc: 0.8147
Epoch 3/30
10848/10848 [==============================] - 20s 2ms/sample - loss: 0.4350 - acc: 0.8168 - val_loss: 0.4381 - val_acc: 0.8131
Epoch 4/30
10848/10848 [==============================] - 20s 2ms/sample - loss: 0.4310 - acc: 0.8169 - val_loss: 0.4276 - val_acc: 0.8151
Epoch 5/30
10848/10848 [==============================] - 20s 2ms/sample - loss: 0.4219 - acc: 0.8180 - val_loss: 0.4250 - val_acc: 0.8157
Epoch 6/30
10848/10848 [==============================] - 20s 2ms/sample - loss: 0.4173 - acc: 0.8187 - val_loss: 0.4198 - val_acc: 0.8157
Epoch 7/30
10848/10848 [==============================] - 20s 2ms/sample - loss: 0.4138 - acc: 0.8198 - val_loss: 0.4

In [53]:
# evaluate on test set
print("------------Performance------------")
y_predict = lstm_model.predict(X_test_inputs_tensor)
# y_pred_for_binary = y_predict > 0.5
# y_pred_for_binary = y_pred_for_binary.astype(int)
for i, col in enumerate(y_test_cohort_df):
    print("AUC score:", roc_auc_score(y_test_cohort_df[col], y_predict[:, i]))

------------Performance------------
AUC score: 0.8039577252612748
AUC score: 0.8881260652819887
AUC score: 0.8226530396781112
AUC score: 0.7051721744849084
AUC score: 0.7725852814835819
AUC score: 0.691077710181076
AUC score: 0.7093184535060741
AUC score: 0.7471941034478308
AUC score: 0.7844623173097609
AUC score: 0.8580401206490225
AUC score: 0.7083981948834064
AUC score: 0.6282040920240688
AUC score: 0.7309370334777852
AUC score: 0.6729834478814641
AUC score: 0.7203386213604333
AUC score: 0.7858263377652662
AUC score: 0.7535313578111175
AUC score: 0.7942979134263213
AUC score: 0.692334234605867
AUC score: 0.7536572150984286
AUC score: 0.6628565773116507
AUC score: 0.7939877285512362
AUC score: 0.876393053435876
AUC score: 0.824642251715058
AUC score: 0.8155584021525999
AUC score: 0.8388321764439961
AUC score: 0.8241021521695675


In [54]:
# https://stackoverflow.com/questions/42202872/how-to-convert-list-to-row-dataframe-with-pandas
test_cohort = cohort[cohort.split == 'test']
## evaluate your model on test_cohort here
## for each field `i` in targets, write your predictions into a new column called 'pred' + i

pred_name = ['predAcute Renal',
       'predCerebrovascular', 'predMyocardial', 'predDysrhythmias',
       'predChronic Kidney', 'predCOPD', 'predComp. Surgical',
       'predConduction', 'predHeart Failure', 'predAtherosclerosis',
       'predDiabetes Comp', 'predDiabetes No Comp', 'predLipid Metabolism',
       'predHypertension', 'predFluid Disorder', 'predGI Hemorrhage',
       'predHypertension Comp', 'predOther Liver', 'predLower Resp',
       'predUpper Resp', 'predPleurisy', 'predPneumonia', 'predResp Failure',
       'predSepticemia', 'predShock', 'predAny Acute', 'predAny Chronic']

for i, item in enumerate(pred_name):
    test_cohort[item] = y_predict[:, i]
#     test_cohort['a'] = y_pred_for_binary[:, 5]

In [55]:
import keras
from keras.models import load_model

# save model
lstm_model.save('model.h5')  # creates a HDF5 file 'my_model.h5'
# torch.save(lstm_model, open(os.path.join('model.pt'), 'wb'))

# load model
# lstm_model = load_model('model.h5')

Using TensorFlow backend.


In [57]:
# 5b table
# Run this code and paste the table into your report
aucs = []
for i in targets:
    aucs.append((i, roc_auc_score(test_cohort[i], test_cohort['pred'+i]),
                log_loss(test_cohort[i], test_cohort['pred'+i]),
                average_precision_score(test_cohort[i], test_cohort['pred'+i])))
aucs.append(('Mean', np.mean([i[1] for i in aucs]),np.mean([i[2] for i in aucs]),np.mean([i[3] for i in aucs])))
res = pd.DataFrame(aucs, columns = ['Task','AUROC', 'logloss', "AUPRC"])
res.style.format({
    'AUROC': '{:.3f}',
    'logloss': '{:.3f}',
    'AUPRC': '{:.3f}'
})

,Task,AUROC,logloss,AUPRC
0,Acute Renal,0.804,0.452,0.576
1,Cerebrovascular,0.888,0.218,0.580
2,Myocardial,0.823,0.310,0.442
3,Dysrhythmias,0.705,0.595,0.552
4,Chronic Kidney,0.773,0.298,0.309
5,COPD,0.691,0.363,0.250
6,Comp. Surgical,0.709,0.557,0.501
7,Conduction,0.747,0.209,0.170
8,Heart Failure,0.784,0.488,0.601
9,Atherosclerosis,0.858,0.432,0.770


In [59]:
# 5c - just run the code and paste the table
import metrics
importlib.reload(metrics)
cohort = cohort.reset_index()
test_cohort = test_cohort.reset_index()
cohort['intersection'] = test_cohort.apply(lambda x: x['gender'] + ' ' + x['ethnicity'], axis = 1)
test_cohort['intersection'] = test_cohort.apply(lambda x: x['gender'] + ' ' + x['ethnicity'], axis = 1)
raw_samples = {t: {} for t in targets}
for i in tqdm(range(500)):    
    bootstrap = test_cohort.sample(frac = 1, replace = True)
    for t in targets:
        for v in ('insurance', 'ethnicity', 'gender', 'language', 'intersection'):
            gaps = metrics.recall_gap_multigroup(bootstrap, v, t, 'pred' + t, 0.3)
            for g in gaps:
                gname = v+'__'+g
                if gname in raw_samples[t]:
                    raw_samples[t][gname].append(gaps[g])
                else:
                    raw_samples[t][gname] = [gaps[g]]  

In [60]:
df_dict = {}
for task_raw in raw_samples:
    df_dict[task_raw] = {}
    for g in raw_samples[task_raw]:
        avg = np.nanmean(raw_samples[task_raw][g])
        errors = avg-np.array(raw_samples[task_raw][g])
        df_dict[task_raw][g] = avg
        df_dict[task_raw][g+'lower'] = avg-np.nanpercentile(errors, 97.5) 
        df_dict[task_raw][g+'upper'] = avg-np.nanpercentile(errors, 2.5)

gap_df = pd.DataFrame.from_dict(df_dict, orient = 'index')
sig_dict = {v: {} for v in ('insurance', 'ethnicity', 'gender', 'language', 'intersection') }
for target in gap_df.index:
    for gname in gap_df.columns:
        if not (gname.endswith('lower') or gname.endswith('upper')):
            v,g = gname.split('__')
            if g not in sig_dict[v]:
                sig_dict[v][g] = [0,0]
            if gap_df.loc[target, gname+'lower'] > 0 and gap_df.loc[target, gname+'upper'] > 0:                
                sig_dict[v][g][0]+=1
                sig_dict[v][g][1]+=1
            if gap_df.loc[target, gname+'lower'] < 0 and gap_df.loc[target, gname+'upper'] < 0:                
                sig_dict[v][g][0]+=1               

for v in ('insurance', 'ethnicity', 'gender', 'language', 'intersection') :
    print(v)
    temp = pd.DataFrame.from_dict(sig_dict[v], orient = 'index', columns = ['# Significant', '# Favored'])    
    temp['% Favored'] = (temp['# Favored']/temp['# Significant']).apply(lambda x: '{0:.2f}%'.format(x*100))
    temp['% Pop'] = temp.apply(lambda x: '{0:.2f}%'.format((cohort[v] ==x.name).sum()/cohort.shape[0]*100), axis = 1)
    display(temp.sort_values(by = '# Significant', ascending = False))
    print('\n')

/data/anaconda/envs/ml4h/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """
/data/anaconda/envs/ml4h/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1372: RuntimeWarning: All-NaN slice encountered
  overwrite_input=overwrite_input, interpolation=interpolation


insurance


,# Significant,# Favored,% Favored,% Pop
Medicaid,0,0,nan%,8.34%
Medicare,0,0,nan%,53.97%
Government,0,0,nan%,2.90%
Private,0,0,nan%,33.66%
Self Pay,0,0,nan%,1.13%




ethnicity


,# Significant,# Favored,% Favored,% Pop
white,0,0,nan%,70.05%
hispanic,0,0,nan%,3.15%
other,0,0,nan%,16.88%
black,0,0,nan%,7.61%
asian,0,0,nan%,2.30%


...

,# Significant,# Favored,% Favored,% Pop
F,0,0,nan%,43.14%
M,0,0,nan%,56.86%




language


,# Significant,# Favored,% Favored,% Pop
English,0,0,nan%,51.22%
Other,0,0,nan%,8.31%
Missing,0,0,nan%,40.47%




intersection


,# Significant,# Favored,% Favored,% Pop
F asian,1,0,0.00%,0.26%
F white,0,0,nan%,9.10%
M hispanic,0,0,nan%,0.56%
M other,0,0,nan%,2.89%
M white,0,0,nan%,12.03%
M black,0,0,nan%,1.09%
F other,0,0,nan%,1.92%
M asian,0,0,nan%,0.44%
F hispanic,0,0,nan%,0.37%
F black,0,0,nan%,1.16%
